In [3]:
# 셀레니움
from selenium import webdriver 
# 셀레니움 By 클래스 가져오기
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# NoSuchElementException import 추가
from selenium.common.exceptions import NoSuchElementException  
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
import logging
from selenium.common.exceptions import TimeoutException, WebDriverException

In [4]:
# 크롬 웹 드라이버 초기화 및 카카오 맵 페이지로 이동
driver = webdriver.Chrome() 
url = 'https://m.map.kakao.com/'
driver.get(url)

# 검색창 element와 검색창 id 가져오기
search_box = driver.find_element(By.ID,'innerQuery')
# 키워드 입력
search_box.send_keys("종로구 빵")

# Enter 키 누르기
search_box.send_keys(Keys.RETURN)

# 페이지가 로드될 때까지 잠시 대기
time.sleep(3)

# 베너 닫기
driver.find_element(By.XPATH, f'//*[@id="daumWrap"]/div[1]/div/a[2]/span').click()

# 더보기 버튼을 누를 수 있는지 여부 확인하는 함수
def can_click_load_more_button():
    try:
        load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
        return load_more_button.is_enabled() and load_more_button.is_displayed()
    except:
        return False

# 더보기 버튼 클릭 함수
def click_load_more_button():
    load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
    load_more_button.click()
    time.sleep(3)  # 버튼을 클릭한 후에 충분한 시간 대기 (페이지가 로딩될 때까지)

# 더보기 버튼을 누를 수 없을 때까지 반복해서 클릭
while can_click_load_more_button():
    click_load_more_button()
    
# 페이지에 표시된 정보의 갯수 확인
all_results = driver.find_elements(By.XPATH, '//*[@id="placeList"]/li')
print("페이지에 표시된 정보의 갯수:", len(all_results))
# 스크롤 올리기
driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")

페이지에 표시된 정보의 갯수: 76


# 가게 이름 누르고 후기 더보기 누르기

1. 가게 이름 누르고 들어간다
2. 가게 이름 크롤링
3. rate_intro 찾아서 있으면, 4번, 없으면, 7번
4. 첫번째 후기 클릭해서 전체 후기 들어가기
5. 전체 후기 페이지의 더보기 끝 까지 누르기 
6. 뒤로가기 2번
7. 없으면, 뒤로가기 1번

In [ ]:
# 연습 1-3 

# 랜덤한 대기 시간
wait = random.uniform(5,10)
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        logging.info(f"No such element: {xpath}")
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait)
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class



# find_total 수정       
def find_data():
    g = 1  # 루프 밖에서 변수 초기화
    max_attempts = 3  # 최대 시도 횟수를 설정하여 무한 루프 방지
    attempts = 0
    while attempts < max_attempts:
        try:
            xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
            xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
            xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'

            id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
            ID.append(id_element.text)
            time.sleep(wait)

            date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
            Date.append(date_element.text)
            time.sleep(wait)

            try:
                # rv_element를 찾는 시도
                rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
                temp_review_text = rv_element.text.strip()  # 텍스트 내의 공백 제거
                if temp_review_text:  # 리뷰 텍스트가 존재하면
                    Review_text.append(temp_review_text)
                else:  # 리뷰 텍스트가 없는 경우
                    Review_text.append('정보 없음')
            except (NoSuchElementException, TimeoutException):
                # 해당 XPATH의 element를 찾지 못한 경우, 아무것도 하지 않고 넘어감
                Review_text.append('정보 없음')

            driver.execute_script("window.scrollBy(0, 200);")  # 페이지 스크롤 다운
            g += 1  # 다음 요소로 이동
            attempts = 0  # 성공적으로 요소를 찾으면 시도 횟수를 초기화
        except (TimeoutException, NoSuchElementException):
            attempts += 1  # 요소를 찾지 못하면 시도 횟수 증가

        # 현재 스크롤 위치를 확인하여 이전 위치와 비교하는 로직은 제거함
        # 대신, 최대 시도 횟수에 도달하면 반복을 종료
        if attempts >= max_attempts:
            print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
            break 
        
# 연습 1-3

In [6]:
# 연습 1-4

# 랜덤한 대기 시간
wait = random.uniform(5,10)
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 가게이름 클릭 함수
def click_def(index):
    xpath = f'//*[@id="placeList"]/li[{index}]/a[1]/span[2]/span[1]/strong'
    try:
        driver.find_element(By.XPATH, xpath).click()
    # 베너가 있는 경우 다른 XPATH로 시도
    except NoSuchElementException:
        logging.info(f"No such element: {xpath}")
        xpath = f'//*[@id="placeList"]/li[{index}]/a/span[2]/span[1]/strong' 
        try:
            driver.find_element(By.XPATH, xpath).click()
        except NoSuchElementException:
            logging.info(f"No such element: {xpath}")
            return None

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait)
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class

# find_total 수정       
def find_data():
    g = 1  # 루프 밖에서 변수 초기화
    max_attempts = 3  # 최대 시도 횟수를 설정하여 무한 루프 방지
    attempts = 0
    while attempts < max_attempts:
        try:
            xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
            xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
            xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'

            try:
                id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
                temp_ID = id_element.text.strip()
                if temp_ID:
                    ID.append(id_element.text)
                else:
                    ID.append('정보 없음')
            except (WebDriverException or NoSuchElementException or TimeoutException): 
                ID.append('정보 없음')
                # break

            time.sleep(wait)

            try:
                date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
                Date.append(date_element.text)
            except (WebDriverException or NoSuchElementException or TimeoutException): 
                # Date.append('정보 없음')
                break

            time.sleep(wait)

            try:
                rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
                temp_review_text = rv_element.text.strip()
                if temp_review_text:
                    Review_text.append(temp_review_text)
                else:
                    Review_text.append('정보 없음')
            except (WebDriverException or NoSuchElementException or TimeoutException):  
                Review_text.append('정보 없음')

            driver.execute_script("window.scrollBy(0, 200);")
            g += 1
            attempts = 0
        # 이 부분은 모든 요소에 대한 크롤링 시도가 실패했을 때를 처리
        except (WebDriverException or NoSuchElementException or TimeoutException):  
            attempts += 1

        if attempts >= max_attempts:
            print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
            break
        
# 연습 1-4

In [ ]:
# webdriverexception 처리하기- 완성
# find_data 함수에 넣어 수정할 것
wait = random.uniform(5,10)
ID = []
Date = []
Review_text = []
g = 1  # 후기 번호 1부터 시작
max_attempts = 3  # 최대 시도 횟수를 설정하여 무한 루프 방지
attempts = 0

while attempts < max_attempts:
    try:
        xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
        xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
        xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'

        try:
            id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
            ID.append(id_element.text)
        except (WebDriverException or NoSuchElementException or TimeoutException): 
            # ID.append('정보 없음')
            break

        time.sleep(wait)

        try:
            date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
            Date.append(date_element.text)
        except (WebDriverException or NoSuchElementException or TimeoutException): 
            # Date.append('정보 없음')
            break

        time.sleep(wait)

        try:
            rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
            temp_review_text = rv_element.text.strip()
            if temp_review_text:
                Review_text.append(temp_review_text)
            else:
                Review_text.append('정보 없음')
        except (WebDriverException or NoSuchElementException or TimeoutException):  
            Review_text.append('정보 없음')

        driver.execute_script("window.scrollBy(0, 200);")
        g += 1
        attempts = 0
    # 이 부분은 모든 요소에 대한 크롤링 시도가 실패했을 때를 처리
    except (WebDriverException or NoSuchElementException or TimeoutException):  
        attempts += 1

    if attempts >= max_attempts:
        print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
        break


In [20]:
len(Review_text)

24

In [7]:
# 연습 2번
store = []
ID = []
Date = []
Review_text = []
# for i in range(5, len(all_results) + 1):
for i in range(5, 7):
    # 배너 넘기기
    if i == 3:
        continue
    
    # 2~5초 랜덤한 대기 시간
    wait_time = random.uniform(1, 5)
    # 가게 이름 클릭
    click_def(i)
    time.sleep(wait_time)
    # 가게 이름 크롤링
    cw = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text
    # store.append(cw)
    time.sleep(wait_time)
    
    # 스크롤 내릴 위치 찾기
    try:
        more_reviews_link = driver.find_element(By.CLASS_NAME, 'rate_intro')
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 후기 더보기 링크가 보이도록 스크롤
    scroll_to_element(more_reviews_link)
    # time.sleep(wait_time)
    
    # 첫번째 리뷰누르기 - 전체 리뷰 페이지로 넘어가기
    try:
        driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/ul[*]/li[1]/div[1]/div[2]/p').click()
    except NoSuchElementException:
        driver.back()
        continue
    time.sleep(wait_time)
    
    # 첫 번째 더보기 버튼 클릭
    class_click()
 
    # 스크롤 올리기
    driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")

    find_data()
    temp_store = [cw for _ in range(len(ID)-1)]
    store.append(temp_store[0])
    temp_store = 0
    
    # 검색 결과 페이지로 돌아가기
    driver.back()
    driver.back()
    
    # 페이지가 로드될 때까지 잠시 대기
    time.sleep(wait_time)

Class changed. Stopping click.
Class changed. Stopping click.


In [8]:
print(len(store))
print(len(ID))
print(len(Date))
print(len(Review_text))

2
718
716
716


In [49]:
total_jongno

,Store,ID,Date,Review_text
0,스코프 서촌점,시리얼,2024.03.30.,예전에 스콘 먹고 감동했던 기억이 있어 다시 방문했는데 조금 실망입니다. 너무 달아...
1,스코프 서촌점,쩝문가,2024.03.23.,명성에 비해 스콘이나 타 디저트 애매합니다. \n유명하지 않았던 평타라 생각하지만\...
2,스코프 서촌점,박현아,2024.03.01.,정보 없음
3,스코프 서촌점,별점조작은말없이1점,2024.02.25.,갠적으로 스콘은 질기고 걍평범하고 맛없고 아아 싫어서 차시켰는데 차도 너무 걍 싸구...
4,스코프 서촌점,고영,2024.02.20.,다 맛있다
...,...,...,...,...
1278,런던베이글뮤지엄 안국점,Im___soo,2021.10.06.,Assorted Bagel & Cream Cheese
1279,런던베이글뮤지엄 안국점,도리도리,2021.10.05.,줄이 너무 길고 아침시간에 못가면 베이글이 없네요...
1280,런던베이글뮤지엄 안국점,wa,2021.09.27.,가격이 ㅠㅠ 한번으로 족한…
1281,런던베이글뮤지엄 안국점,sqzplay,2021.09.21.,근데 위생은 조금만 신경 써주셨음 좋겠음... 기분 좋게 커피 다 마시다 밑 바닥에...


In [53]:
total_jongno['Review_text'][1]

'명성에 비해 스콘이나 타 디저트 애매합니다. \n유명하지 않았던 평타라 생각하지만\n기대 이하라고 생각드네요.'